# **Task 1 – Data Preprocessing**

The two main goals of this project are to:

**1**. Develop an ARIMA model to predict ADR

**2**. Develop three distinct classification models (logistic regression, random forest, and support vector machine (SVM) to predict whether a hotel booking will be cancelled or not.

Consequently, data preprocessing is necessary before the model development phase to ensure the accuracy of the dataset.

In [1]:
#Import data manipulation library
import pandas as pd

#Filter/Ignore warnings
from warnings import filterwarnings
filterwarnings('ignore')

The dataset, ‘hotel booking demand’ ('hotel'), contains booking information for two hotels, city hotel and resort hotel, in Portugal. The arrival dates for the bookings are between 1st July 2015 and 31st August 2017 (Nuno, Almeida and Nunes, 2019).

In [2]:
#Load dataset
df = pd.read_csv ('hotel.csv')

In [3]:
#Evaluate initial shape of data
df.shape

(119390, 32)

* The response variable for the classification models is identified as ‘is_canceled’. It represents whether a booking will be cancelled (1) or not (1). Examining the first few rows of the dataframe reveals that the column ‘reservation_status’ contains information that directly reflects the ‘is_canceled’ column. To prevent data leakage and ensure model accuracy this column is dropped.

* In addition, the ‘reservation_status_date’ column is also dropped as it just represents the date when the reservation status was last updated, and thus, does not contain any predictive information about cancellations. Potential data leakage issues are also avoided by dropping this column.

* Lastly, it can also be seen that some columns, such as 'agent' and 'company', have missing ('NaN') values. These need to be further investigated before the data modelling phase.

In [4]:
#Examine head
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [5]:
#Drop 'reservation_status' and 'reservation_status_date'
df.drop(['reservation_status'], inplace = True, axis = 1)
df.drop(['reservation_status_date'], inplace = True, axis = 1)
df.shape

(119390, 30)

Summary statistics show that:

* The column 'lead_time' exhibits both a high mean (104) and a high standard deviation (106), indicating considerable variability in the duration between the booking date and the arrival date. However, a high standard deviation in lead time is not inherently indicative of outliers, as a wide range of lead times is typical in the hospitality industry. Nonetheless, the presence of a maximum lead time of 737 highlights significant differences between booking and arrival dates. Large durations between booking and arrival may introduce uncertainties and may potentially increase the probability of cancellations.

* The minimum, 25%, 50%, and 75% values in the columns 'previous_cancellations', 'previous_bookings_not_canceled', and 'days_in_waiting_list' are zero, though the maximum values for these columns are 26, 72, and 391 respectively. This suggests that these columns are heavily skewed towards zero, indicating that the majority of customers have not had any previous cancellations or non-canceled bookings or long wait times. However, the presence of relatively high maximum values suggests the existence of outliers. Further analysis is required to understand the nature of these outliers and determine whether they represent genuine instances of customer behaviour or if they are the result of data entry errors.

* The mean (0.22) and standard deviation (0.65) of 'booking_changes' suggest low variation. However, the presence of a maximum value of 21 in this column indicates instances of considerable instability in bookings. Continuous changes to bookings might heighten the likelihood of cancellations.

* The mean (101) and standard deviation (50) of the ‘adr’ column are relatively high, which suggests that there are likely outliers or extreme values influencing the spread and distribution of ADR. The minimum ADR value is -6.38, and a negative value indicates that the data may be erroneous as ADR is typically a non-negative value. Additionally, the maximum ADR value is 5,400, which does indeed suggest the presence of outliers. Further analysis and data cleansing may be necessary to address these issues and ensure data accuracy.

In [6]:
#Examine summary statistics
df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103890,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


Examining unique values for the categorical columns reveals that 'country' 'nan' values as well. Furthermore, this column also has a lot of unique values. Including this variable in the models could, therefore, be problematic as the presence of numerous unique values increases dimensionality, which makes modelling more complex and increases the risk of overfitting. Moreover, if certain countries have relatively few occurrences compared to others, then the models will struggle to capture meaningful relationships associated with values that have limited representation. Data exploration is necessary to determine whether to include this column in the models or not.

In [7]:
#Examine unique values for categorical columns
for column in df.columns:
    if df[column].dtype == 'object':
        print("\033[1m" + column + "\033[0m")
        print(df[column].unique())
        print("\n")

hotel
['Resort Hotel' 'City Hotel']


arrival_date_month
['July' 'August' 'September' 'October' 'November' 'December' 'January'
 'February' 'March' 'April' 'May' 'June']


meal
['BB' 'FB' 'HB' 'SC' 'Undefined']


country
['PRT' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' nan 'ROU' 'NOR' 'OMN' 'ARG' 'POL'
 'DEU' 'BEL' 'CHE' 'CN' 'GRC' 'ITA' 'NLD' 'DNK' 'RUS' 'SWE' 'AUS' 'EST'
 'CZE' 'BRA' 'FIN' 'MOZ' 'BWA' 'LUX' 'SVN' 'ALB' 'IND' 'CHN' 'MEX' 'MAR'
 'UKR' 'SMR' 'LVA' 'PRI' 'SRB' 'CHL' 'AUT' 'BLR' 'LTU' 'TUR' 'ZAF' 'AGO'
 'ISR' 'CYM' 'ZMB' 'CPV' 'ZWE' 'DZA' 'KOR' 'CRI' 'HUN' 'ARE' 'TUN' 'JAM'
 'HRV' 'HKG' 'IRN' 'GEO' 'AND' 'GIB' 'URY' 'JEY' 'CAF' 'CYP' 'COL' 'GGY'
 'KWT' 'NGA' 'MDV' 'VEN' 'SVK' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL' 'SEN'
 'SYC' 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MKD' 'MYS' 'ARM' 'JPN' 'LKA' 'CUB'
 'CMR' 'BIH' 'MUS' 'COM' 'SUR' 'UGA' 'BGR' 'CIV' 'JOR' 'SYR' 'SGP' 'BDI'
 'SAU' 'VNM' 'PLW' 'QAT' 'EGY' 'PER' 'MLT' 'MWI' 'ECU' 'MDG' 'ISL' 'UZB'
 'NPL' 'BHS' 'MAC' 'TGO' 'TWN' 'DJI' 'STP' 'KNA' 'ET

In [8]:
#Check for missing values
print(df[df.columns[df.isnull().any()]].isnull().sum())
print((df[df.columns[df.isnull().any()]].isnull().mean() * 100).round(2))

children         4
country        488
agent        16340
company     112593
dtype: int64
children     0.00
country      0.41
agent       13.69
company     94.31
dtype: float64


The ‘company’ column has a high percentage of missing values that can’t be reasonably imputed so it is dropped.

In [9]:
#Drop 'company'
df.drop(['company'], inplace = True, axis = 1)
df.shape

(119390, 29)

The four (4) missing values in the ‘children’ column are imputed with zero as it is reasonable to assume that if ‘children’ is left empty then the guests making the booking do not have children.

In [10]:
#Impute missing values in 'children' with zero (0)
df['children'].fillna(0, inplace = True)
df['children'].isnull().sum()

0

Further investigation of the ‘agent’ column shows that the values likely serve as unique identifiers for specific agents/agencies. Consequently, this column is also dropped since values can’t be imputed without introducing bias (favouring agents that appear more often) and because the high number of unique values would lead to the classification models overfitting to the data.

In [11]:
#Investigate missing values in 'agent'
df['agent'].head(20)

0       NaN
1       NaN
2       NaN
3     304.0
4     240.0
5     240.0
6       NaN
7     303.0
8     240.0
9      15.0
10    240.0
11    240.0
12    240.0
13    241.0
14    241.0
15    240.0
16      8.0
17    240.0
18      NaN
19    250.0
Name: agent, dtype: float64

In [12]:
#Drop 'agent'
df.drop(['agent'], inplace = True, axis = 1)
df.shape

(119390, 28)

Lastly, for the ‘country’ column, as there is a low percentage of missing values (0.41), they can be imputed with the mode of the column.

In [13]:
#Mode of 'country'
df['country'].mode()

0    PRT
Name: country, dtype: object

In [14]:
#Impute missing values in 'country' with mode (PRT)
df['country'].fillna('PRT', inplace = True)
df['country'].isnull().sum()

0

Finally, examination of data types across all columns shows that ‘children’ and ‘adr’, numeric columns, are type ‘float64’ when the others are ‘int64’. To enhance data consistency, the data type for these two columns is converted to ‘int64’.

In [15]:
#Examine datatype of all columns
df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
days_in_waiting_list              

In [16]:
#Convert 'float64' type columns to 'int64'
df['children'] = df['children'].astype(int)
df['adr'] = df['adr'].astype(int)
df.dtypes

hotel                             object
is_canceled                        int64
lead_time                          int64
arrival_date_year                  int64
arrival_date_month                object
arrival_date_week_number           int64
arrival_date_day_of_month          int64
stays_in_weekend_nights            int64
stays_in_week_nights               int64
adults                             int64
children                           int64
babies                             int64
meal                              object
country                           object
market_segment                    object
distribution_channel              object
is_repeated_guest                  int64
previous_cancellations             int64
previous_bookings_not_canceled     int64
reserved_room_type                object
assigned_room_type                object
booking_changes                    int64
deposit_type                      object
days_in_waiting_list               int64
customer_type   

In [17]:
#Download the updated dataset
df.to_csv('hotel_dataset.csv', index = False)